In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
import numpy as np


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Quadro M6000 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN not available)


In [2]:
filename = "trump_speech.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
chars = set(raw_text)

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


maxlen = 100
step = 1
sentences = []
next_chars = []
for i in range(0, len(raw_text) - maxlen, step):
    sentences.append(raw_text[i: i + maxlen])
    next_chars.append(raw_text[i + maxlen])
print('nb sequences:', len(sentences))



X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

('nb sequences:', 916951)


In [8]:
model = Sequential()
model.add(LSTM(512,  input_shape=(100,69), return_sequences=True))
model.add(Dropout(0.20))

# model.add(LSTM(512, return_sequences=True))
# model.add(Dropout(0.20))

# model.add(LSTM(512, return_sequences=True))
# model.add(Dropout(0.20))

model.add(LSTM(256, return_sequences=False))
model.add(Dropout(0.20))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))

# compile or load weights then compile depending


In [13]:
# "This above all: to thine own self be true". - Hamlet (Act I, Scene III).
def generatetext(i):
    seed_text = "Yet Hillary Clinton, for months, and despite so many attacks, repeatedly refused to even say the words radical Islam until I challenged her yesterday.".lower()
    generated = '' + seed_text[-100:]
    print(i,"Seed",seed_text)
    for iteration in range(300):
            # create x vector from seed to predict off of
            x = np.zeros((1, 100, len(chars)))
            for t, char in enumerate(seed_text[-100:]):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = np.argmax(preds)
            next_char = indices_char[next_index]

            generated += next_char
            seed_text = seed_text[1:] + next_char
#             print seed_text,next_char
    print('follow up with: ' + generated)

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

In [15]:
# model.load_weights('One_go_larger_network_0.h5')

In [17]:
epochs = 30
for i in range(12,epochs):
    model.fit(X,y, batch_size=1000,nb_epoch=1)
    model.save_weights('trump_layers'+str(i)+'.h5')
    generatetext(i)

Epoch 1/1
916951/916951 [==============================] - 1124s - loss: 1.0908 - acc: 0.6644  
(12, 'Seed', 'yet hillary clinton, for months, and despite so many attacks, repeatedly refused to even say the words radical islam until i challenged her yesterday.')
follow up with: ny attacks, repeatedly refused to even say the words radical islam until i challenged her yesterday. the world that we have to do it and the world that we have to do it. i don’t know what they don’t want to be a lot of money. i don’t know what they don’t want to be a lot of money. i don’t know what they don’t want to be a lot of money. i don’t know what they don’t want to be a lot 
Epoch 1/1
916951/916951 [==============================] - 1125s - loss: 1.0740 - acc: 0.6690  
(13, 'Seed', 'yet hillary clinton, for months, and despite so many attacks, repeatedly refused to even say the words radical islam until i challenged her yesterday.')
follow up with: ny attacks, repeatedly refused to even say the words radi

916951/916951 [==============================] - 1125s - loss: 0.9593 - acc: 0.7003  
(24, 'Seed', 'yet hillary clinton, for months, and despite so many attacks, repeatedly refused to even say the words radical islam until i challenged her yesterday.')
follow up with: ny attacks, repeatedly refused to even say the words radical islam until i challenged her yesterday. the worst thing that i was a great company and i was a great company and i was a great company and i was a great company and i was a great company and i was a great company and i was a great company and i was a great company and i was a great company and i was a great company and i was a great comp
Epoch 1/1
916951/916951 [==============================] - 1125s - loss: 0.9521 - acc: 0.7024  
(25, 'Seed', 'yet hillary clinton, for months, and despite so many attacks, repeatedly refused to even say the words radical islam until i challenged her yesterday.')
follow up with: ny attacks, repeatedly refused to even say the word

In [ ]:
generatetext(i)
